In [44]:
# Basic
from threading import Thread
import os
import time
import datetime
import socket
import json

# Plotting and Data
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

# IPYthon
from IPython.display import clear_output
from IPython.display import IFrame
import ipywidgets as iw

#Debugging
import logging
logging.basicConfig(level=logging.WARNING)

In [45]:
### EEGStream Class ###

class EEG:
    '''The following variables are shared between a streaming and a plotting instance
    of EEG. They run in different threads and need to access a shared variable space.'''
    
    # Data Structure
    n_channels = 8
    time = {"time":pd.Series([],dtype="int64")}
    channels = {str(i):pd.Series([],dtype="float64") for i in range(n_channels)}
    rate = {"sampling_rate":pd.Series([],dtype="float64")}
    data = pd.DataFrame({**time, **channels, **rate})
    plot_buffer = data
    
    # Preview of Data columns              #
    # time   | 0  | 1 | .. | sampling_rate # 
    
    # GUI editable parameters
    connected = False
    recording = False
    plotting = False
    active_channels = [1,2,3,4,5,6,7,8]
    
    time_window = 2
    mV_window = 2000
    sampling_rate = 222
    fps = 25
    
    def to_csv(self):
            if 'sessions' not in os.listdir():
                os.mkdir('sessions')
            time = datetime.datetime.now().strftime("%Y-%m-%d-%H%M")
            filepath = f"sessions/{time}.csv"
            EEG.data.to_csv(filepath)
            print(f"Data saved to {filepath}")
            return
        
    class Streamer: 
        
        def __init__(self):
            self.socket = None
            self.connection = None 

        def build_connection(self,host="",port=65432):
            '''
            Creates a UDP Socket and waits for a connection with the specified parameters.
            returns
            ss : streaming socket
            connection 
            '''
            ss = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

            try:
                ss.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
                ss.bind((host, port))
            except socket.error as e:
                print(str(e))

            print("Waiting for a Connection..")
            ss.listen()  
            connection, address = ss.accept()

            ip = address[0]
            port = address[1]
            print(f"Connected to {ip}:{port}")

            self.socket = ss, self.connection = connection
            EEGStream.connected = True
            return

        def record(self, socket, connection, buffersize=1):
            '''
            Builds a connection, reads data from it and creates packages in the form of dictionaries.
            Expects incoming data to be stringified json files. 
            ''' 
            connection = self.connection
            socket = self.socket
            pkg = ""        # 1 pkg: 1 json object
            pkg_count = 0
            empty_msgs = 0  # counter

            start_time = time.time()
            sampling_rate = 0 

            while True:  
                # Receive data (buffer length=1byte)
                response = connection.recv(buffersize).decode("utf-8")   

                # Count Empty Responses
                if not response: 
                    empty_msgs += 1  
                    if empty_msgs > 1000: break

                # Beginning of Package
                if "{" in response:  
                    pkg += response[response.index("{"):]  # add everything after first curly
                    empty_msgs = 0   # reset counter

                # Content
                elif "}" not in response: 
                    pkg += response

                # End of Package 
                else:

                    pkg_count += 1
                    pkg += response

                    elapsed = (time.time() - start_time)
                    json_string = pkg.rstrip("\n").strip() # remove ending break lines and white spaces
                    current = json.loads(json_string)  # parse stringified JSON as dict

                    sampling_rate = int(pkg_count / elapsed) if pkg_count > 1 else 0

                    self.store(elapsed,current,sampling_rate)  # add current read package for plotting 
                    empty_msgs = 0
                    pkg = pkg[pkg.index("}")+1:]  # only keep data after the end of the first package

                    if pkg_count > self.xwindow+20:
                        recording=True

            socket.close()  # disconnect listening socket
            logging.info("Ended Streaming Session")
            recording = False
            
        def store(self, elapsed, current, sampling_rate):
            '''
            Rearranges incoming data into a dictionary of the following form
            '''
            xwindow = self.xwindow

            original_time = current["time"]
            del current["time"]
            datadic = {**{'time':elapsed},
                       **current,
                       **{'sampling_rate':sampling_rate}
                      }

            EEGStream.data = EEGStream.data.append(datadic, ignore_index=True)

            if EEGStream.data.shape[0] > xwindow+20:
                EEGStream.plot_buffer = self.data[-xwindow]
   
    class Plotter:
        
        def __init__(self):
            '''Feel free to change the time_window,fps and pkg_limit. Other changes probably break the programm'''
            self.xwindow = int(EEGStream.sampling_rate*EEGStream.time_window) # Expected number of simultaneously plotted data
            # Plotting Instance
            self.fig = None
            
        def live_plot(self):
            '''
            Starts a thread that records data from the socket and shows a live plot
            '''

            logging.info("Waiting for incoming data..")
            while not EEGStream.recording:
                time.sleep(0.5)

            self.build_plot()
            self.animate_plot()

            if input("Stream Again? | 'y' for 'yes'") == 'y':
                self.live_plot()
                clear_output()
            else:
                return

            recording_thread.join() # Wait for the thread to end properly

                                    #=== PLOT RELATED FUNCTIONS ===#

        def build_plot(self):
            # Preparing the Plot and extracting the lines
            plot = EEGStream.data.plot(x="time", y= [str(channel+1) for channel in range(EEGStream.n_channels)])
            self.fig = plott.gcf()
            self.lines = [line for line in self.fig.gca().lines]

            # Setting Some Plot properties
            self.fig.gca().set_title("EEG Streaming @ 0 Hz")
            self.fig.gca().set_ylim(-2500,2500)
            self.fig.gca().set_xlabel("time [s]")
            self.fig.gca().set_ylabel("signal [mV]")


        def animate_plot(self):
            global recording

            interval = 1/self.fps
            while recording:
                time.sleep(interval)
                success = self.update_lines()
                self.fig.canvas.draw()

            return 

        def update_lines(self):
            # Update Plot Lines
            for i,line in enumerate(self.lines):
                line.set_data(data.time, data[f"{i+1}"])
            #Update x-axis
            right = data.time.iloc[-1] # last timestamp
            left = right-self.time_window
            self.fig.gca().set_xlim(left,right)
            #Update y-axis
            self.fig.gca().set_ylim(-mV_window,mV_window)
            #Update other Plot elements
            self.fig.gca().set_title(f"Traumschreiber Streaming @ {data.sampling_rate.iloc[-1]}Hz")
            self.fig.gca().legend(loc="upper right")

            return 



In [46]:
# Initialize a Stream
eegstream = EEGStream()

In [83]:
# Initiating GUI:
# Message to user

class GUI:
    def __init__(self):
        self.streamer = EEG.Streamer()
        self.plotter = EEG.Plotter()
        self.gui_elements = []
        self.build_widgets()

    def build_widgets(self):
        instruction = iw.HTML(value= "<h1>Some explanation... ")
        self.gui_elements.append(instruction)

        connect_button = make_button(description="Build Connetion",
                                     target=self.build_connection)
        self.gui_elements.append(connect_button)

        record_button = make_button(description="Start Recording",
                                    target=self.start_recording)
        self.gui_elements.append(record_button)

    def display(self):
        for i in self.gui_elements:
            display(i)

    def build_connection(self,obj):
        if obj["new"]:  
            self.eegstream.build_connection()
        else:
            EEGStream.connected=False
            print(f"Value of EEGStream.connected is now {EEGStream.connected}")
            return

    def start_recording(self,obj):
        if obj["new"]:
            EEGStream.recording = True
            print(f"Value of EEGStream.connected is now {EEGStream.recording}")
        else:
            EEGStream.recording=False
            print(f"Value of EEGStream.connected is now {EEGStream.recording}")
            return

    def start_plotting(self,obj):
        if obj["new"]:
            EEGStream.plotting = True
            print(f"Value of EEGStream.plotting is now {EEGStream.recording}")
        else:
            EEGStream.plotting=False
            print(f"Value of EEGStream.plotting is now {EEGStream.recording}")
            return

    def toggle_value(self, value):
        if obj["new"]:
            value = True
            print("Value is now True")
        else:
            value = False
            print("Value is now False")

    def make_button(description, target, height="50px"): 
        button = iw.ToggleButton(description=description)
        button.layout = iw.Layout(height=height)
        button.observe(target,"value")
        return button


NameError: name 'obj' is not defined

In [59]:
print("test")
time.sleep(4)
clear_output()